In [1]:
import pandas as pd
import numpy as np

In [2]:
meme = pd.read_csv('../data_cleaned/meme_stocks.csv').drop('Unnamed: 0',axis=1)
top = pd.read_csv('../data_cleaned/top_companies.csv').drop('Unnamed: 0',axis=1)

In [7]:
def select_stock_from_csv(df:pd.DataFrame,stock:str,datalist=[])-> pd.DataFrame:
    try:
        if len(datalist) ==0:
            return df[df['tic']==stock]
        else: 
            return df[df['tic']==stock][datalist]
    
    except Exception as e:
        print(f'Data for {stock} is missing')
    

In [8]:
amc = select_stock_from_csv(meme,'AMC',['date','close','volume','tic'])

In [9]:
amc['pct_chng'] = amc['close'].pct_change().fillna(0)

In [10]:
amc['movement'] = amc.apply(lambda x: 1 if x['pct_chng'] > 0 else 0,axis=1)

In [11]:
amc

,date,close,volume,tic,pct_chng,movement
0,2015-01-02,18.900267,420900,AMC,0.000000,0
2,2015-01-05,18.413568,246500,AMC,-0.025751,0
4,2015-01-06,18.561052,271500,AMC,0.008010,1
6,2015-01-07,18.863400,175400,AMC,0.016289,1
8,2015-01-08,19.047752,192800,AMC,0.009773,1
...,...,...,...,...,...,...
3974,2021-08-23,36.779999,75319300,AMC,0.068875,1
3979,2021-08-24,44.259998,228489600,AMC,0.203371,1
3984,2021-08-25,43.959999,209271500,AMC,-0.006778,0
3989,2021-08-26,40.310001,109710200,AMC,-0.083030,0


In [2]:
reddit_20150101 = pd.read_csv('../data/Reddit Data/2015/Reddit_Comment_2015-01.csv')


In [14]:
reddit_20150101 = reddit_20150101.rename(columns={'Body':'text'})
reddit_20150101['label'] = ''

In [15]:
reddit_20150101.to_csv('doccano_demo.csv')

In [16]:
## Score just means upvotes or downvotes

,Date,ID,Parent,text,Score,label
0,15-01-01,cnasafn,t1_cnardcl,Absolutely liquidate. Our bank's PB is not in ...,3,
1,15-01-01,cnase68,t3_2qxx1u,ARCP,7,
2,15-01-01,cnasd41,t1_cnaqx6u,[deleted],1,
3,15-01-01,cnaslz4,t1_cnarxn9,Thanks for the detailed post. Perhaps my tone ...,1,
4,15-01-01,cnasm73,t1_cnal0pd,The messenger used frequently in Asian countries?,4,
...,...,...,...,...,...,...
34582,15-01-31,co71c04,t1_co70tmj,"I prefer puts too, but in the case of SHAK, th...",2,
34583,15-01-31,co71cde,t3_2ubj7y,what the fuck?,2,
34584,15-01-31,co71g4j,t1_co6zv37,"problem is, it seems unlikely you're pulling d...",5,
34585,15-01-31,co71ii6,t1_co6y6d1,"This. You have to go through a brokerage, mo...",3,


In [49]:
import re
from w3lib.html import replace_entities ## removes html entities
import reticker ## extracting tickers
import glob

In [4]:
def replace_ticker(text):
    tickers_to_replace = [i if i in tickers['tickered'].to_list() else '' for i in re.findall(r'\$\w+',str(text))]
    for i in tickers_to_replace:
        if i:
            name = tickers[' name'].loc[tickers['tickered'] == i].iloc[0]
            if name:
                text = text.replace(i, f'{name},')
            else: 
                pass
        else:
            pass
    text = re.sub(r'\$\w+','', str(text))
    return text

In [23]:
len(reddit_20150101.loc[16].Body)#[['Body','Score']]

1179

In [ ]:
maybe create something with a meaningful score?

In [ ]:
## remove \n\n values
## remove &amp;
## remove deleted

In [136]:
def clean_text(text):
    text = str(text)
    text = replace_entities(text)
    text = re.sub(r'\n', '',text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\[removed\]', '',text)
    text = re.sub(r'\[deleted\]', '',text)
    text = re.sub(r'/u/', '',text)

    return text

def generate_tickers(text):
    return reticker.TickerExtractor().extract(str(text))
    

In [109]:
reddit_20150101['clean_text'] = reddit_20150101['Body'].apply(clean_text)
reddit_20150101['tickers'] = reddit_20150101['Body'].apply(generate_tickers)


In [117]:
## create pipeline

def merge_and_clean_data(data_folder):
    main_folder_list = glob.glob(data_folder + '/*')
    final = []
    for year_folders in main_folder_list:
        csv_list = glob.glob(year_folders + '/*.csv')
        for csv in csv_list:
            print(f'loading for {csv}...')
            if csv.split('_')[1] == 'Submission':
                df = pd.read_csv(csv).dropna()
                df['Body'] = df.apply(lambda x: str(x['Title']) + str(x['Text']),axis=1)
                df['text'] = df['Body'].apply(clean_text)
                df['tickers']=df['Body'].apply(generate_tickers)
                df=df[['Date','text','Score','tickers']]
                df = df[df['tickers'].map(len)!=0]
                df = df.rename(columns = {'Date':'date','Score':'score'})
                final.append(df)
                
            if csv.split('_')[1] == 'Comment':
                df = pd.read_csv(csv).dropna()
                df['text'] = df['Body'].apply(clean_text)
                df['tickers']=df['Body'].apply(generate_tickers)
                df=df[['Date','text','Score','tickers']]
                df = df[df['tickers'].map(len)!=0]
                df = df.rename(columns = {'Date':'date','Score':'score'})
                final.append(df)
            
    return pd.concat(final)
    

In [118]:
data_folder = '../data/Reddit data'

In [119]:
output = merge_and_clean_data(data_folder)

loading for ../data/Reddit data/2015/Reddit_Submission_2015-09.csv...
loading for ../data/Reddit data/2015/Reddit_Submission_2015-08.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-09.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-08.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-06.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-12.csv...
loading for ../data/Reddit data/2015/Reddit_Submission_2015-03.csv...
loading for ../data/Reddit data/2015/Reddit_Submission_2015-02.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-07.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-11.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-05.csv...
loading for ../data/Reddit data/2015/Reddit_Submission_2015-01.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-04.csv...
loading for ../data/Reddit data/2015/Reddit_Comment_2015-10.csv...
loading for ../data/Reddit data/2015/Reddit_Sub

In [123]:
import nltk

from nltk.tokenize import TweetTokenizer

In [124]:
tokenizer = TweetTokenizer()

In [142]:
cleaned_data_1 = output[output['score']>= 10]

In [147]:
cleaned_data_1.to_csv('../data_cleaned/reddit_data_19102021.csv')

In [146]:
cleaned_data_1

,date,text,score,tickers
13,15-09-01,US Futures pointing lower... PANIC!!!Well shuc...,14,"[US, PANIC, RIP]"
17,15-09-01,Indisputable proof that the shemitah is upon u...,20,[US]
29,15-09-01,"Dear mods, FS Comeau here with a message for y...",48,[FS]
33,15-09-01,World markets sell off after China PMI and com...,38,[PMI]
37,15-09-01,"Q: VXX / XIV, what's the big deal? A: it's the...",10,"[VXX, XIV, VIX]"
...,...,...,...,...
6681,16-09-30,<---- = # of pussies in this subreddit who pla...,623,[JNUG]
6699,16-09-30,Is it too late to buy-in on AMD?Did I miss the...,11,[AMD]
6708,16-09-30,Lets Start a FundIt can be called WSB on the N...,18,"[JNUG, DUST, AMD, MAST, NVAX, SPY, CALLS, NUGT]"
6728,16-09-30,Building Custom Stock Alerts in Google Spreads...,52,"[UBS, MBA, SMS, X, PM]"


In [ ]:
## dropnas 
## remove [removed]
## [deleted]
## remove links 
## Remove bot messages that remove text